# ファイルのペアリスト作成
ファイルのペアリストを作成します．
- 録音環境による照合制度の変化
  - パソコン録音かPC録音か=`pc`or`sp`


`pc` or `sp`の条件ごとにリストを作成する．
- 有効話者
  - 各条件ごとに2つ以上発話がある
  - `sp`か`pc`かで発話内容が同一
  - `ymgt-A`, `ymgt-B`, `ymgt-C`, `ymgt-D`, `hre_A`, `tmng`, `ktg`, `ymgt-E`, `ymgt-F`

# データ詳細
- `ymgt-A`, `ymgt-B`, `ymgt-C`, `ymgt-D`, `hre_A`, `tmng`, `ktg`, `ymgt-E`, `ymgt-F`
  - 男6，女3
  - 各条件について，6+7+6+6+3+3+6+6+6=49発話

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import itertools

## 各サブディレクトリの音声リスト作成


In [7]:
%cd /content/drive/MyDrive/CS_seminar/8th/data/wav
WAV_ROOT = %pwd
LIST_ROOT = "/content/drive/MyDrive/CS_seminar/8th/data/list"

/content/drive/MyDrive/CS_seminar/8th/data/wav


In [5]:
# 有効話者ディレクトリを定義しておく．後で参照する．
VALID_SPEAKERS = ["ymgt_A", "ymgt_B", "ymgt_C", "ymgt_D", "hre_A", "tmng", "ktg", "ymgt_E", "ymgt_F"]

In [8]:
pc = []
sp = []

spk_list = [item for item in os.listdir() if item[0] != '.'] # ".ipynb_checkpoints"を除外
spk_list = list(set(spk_list) and set(VALID_SPEAKERS)) # 取得した話者ディレクトリのリストと，今回参照する話者リストの積を取る．

for spk in spk_list:
  spk_dir = WAV_ROOT + '/' + spk
  subdirs = os.listdir(spk_dir)
  for subdir in subdirs:
    tarsubdir = spk_dir + '/' + subdir
    wavs = os.listdir(tarsubdir)
    if subdir == "pc":
      for item in wavs:
        pc.append(tarsubdir + '/' + item)
    if subdir == "sp":
      for item in wavs:
        sp.append(tarsubdir + '/' + item)


print(len(pc), pc)
print(len(sp), sp)


49 ['/content/drive/MyDrive/CS_seminar/8th/data/wav/ktg/pc/ktg_pc_01.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ktg/pc/ktg_pc_02.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ktg/pc/ktg_pc_03.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ktg/pc/ktg_pc_04.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ktg/pc/ktg_pc_05.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ktg/pc/ktg_pc_06.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/tmng/pc/tm_pc1.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/tmng/pc/tm_pc2.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/tmng/pc/tm_pc3.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ymgt_F/pc/ymgt-F_pc_03.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ymgt_F/pc/ymgt-F_pc_04.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ymgt_F/pc/ymgt-F_pc_01.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ymgt_F/pc/ymgt-F_pc_02.wav', '/content/drive/MyDrive/CS_seminar/8th/data/wav/ymgt_F/

## 各サブディレクトリ内の音声について，組み合わせリストを作成


In [9]:
def is_same_speaker(path1, path2):
  # path文字列を解析して，パス対が同じ話者であるかを判定
  name1 = path1.split('/')[-3]
  name2 = path2.split('/')[-3]
  # # re(正規表現)による実装．今回のpathであればsplit('/')でいいが，複雑な区切り文字とかになると正規表現を用いた方が良い場面も出てくる．
  # dir1 = re.match(r"(.*/).+\.wav", path1).group(1)
  # dir2 = re.match(r"(.*/).+\.wav", path2).group(1)
  return int(name1 == name2)

def gen_2pair_listfile(path_list, outfile_path):
  outfile = open(outfile_path, 'w')
  out_info = {
      "filename": outfile_path,
      "total_wavs": len(path_list),
      "total_pairs": len(path_list) * (len(path_list)-1) // 2,
      "same_pair": 0,
      "diff_pair": 0,
  }
  # itertools.combinations()による実装．
  # 組み合わせをforで構築するのは，`CS_seminar/4th/*/gen_list.py`を参照のこと．
  pairs = list(itertools.combinations(path_list, 2))
  for pair in pairs:
    same_spk = is_same_speaker(*pair)
    outstr = f"{same_spk} {pair[0]} {pair[1]}\n"
    outfile.write(outstr)
    # ペア数を記録
    if same_spk:
      out_info["same_pair"] += 1
    else:
      out_info["diff_pair"] += 1
  outfile.close()
  return out_info

In [10]:
# info = gen_2pair_listfile(pc, f"{LIST_ROOT}/pc.txt")
info = gen_2pair_listfile(pc, f"{LIST_ROOT}/pc.txt")
print(info)
# info = gen_2pair_listfile(sp, f"{LIST_ROOT}/sp.txt")
info = gen_2pair_listfile(sp, f"{LIST_ROOT}/sp.txt")
print(info)


{'filename': '/content/drive/MyDrive/CS_seminar/8th/data/list/pc_new.txt', 'total_wavs': 49, 'total_pairs': 1176, 'same_pair': 117, 'diff_pair': 1059}
{'filename': '/content/drive/MyDrive/CS_seminar/8th/data/list/sp_new.txt', 'total_wavs': 49, 'total_pairs': 1176, 'same_pair': 117, 'diff_pair': 1059}
